# Assembled Notebook — SK Agents (Deployment Diagram)

_Generated from your deployment diagram. Mermaid is **not** embedded in this notebook; only the resulting code is._

In [ ]:
# %% [SETUP]
# Package install (unpin; upgrade SK)
!pip install -U semantic-kernel
!pip -q uninstall -y pydrive2
!pip install agent-framework --pre


In [ ]:
# %% [SETUP-ENV]
# Environment variables for Azure OpenAI (replace placeholder strings with real values)
import os
os.environ["AZURE_OPENAI_ENDPOINT"]    = "https://YOUR-AOAI-ENDPOINT.openai.azure.com"  # TODO
os.environ["AZURE_OPENAI_API_KEY"]     = "YOUR-AOAI-KEY"                                 # TODO
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-10-21"
os.environ["AZURE_OPENAI_DEPLOYMENT"]  = "gpt-35-turbo"                                  # TODO: deployment name
print("Azure env set. Remember to replace placeholders.")

In [ ]:
# %% [KERNEL]
import os
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = Kernel()

service = AzureChatCompletion(
    service_id="azure",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT"),  # your AOAI deployment name
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),           # e.g. https://my-aoai.openai.azure.com
    # api_version is optional (defaults internally); set if you need a specific one:
    # api_version="2024-10-21"
)

kernel.add_service(service)
print("Kernel ready (Azure OpenAI)")

In [ ]:
# %% [TOOLS]
def tool_storage_blob(**kwargs):
    """Blob storage integration placeholder."""
    return {"tool": "storage_blob", "status": "ok", "kwargs": kwargs}

def tool_search_query(**kwargs):
    """Search query (read) placeholder."""
    return {"tool": "search_query", "status": "ok", "kwargs": kwargs}

def tool_search_admin(**kwargs):
    """Search admin (index/skillset ops) placeholder."""
    return {"tool": "search_admin", "status": "ok", "kwargs": kwargs}

def tool_keyvault_secret(**kwargs):
    """Key Vault secret get/set placeholder (DO NOT log real secrets)."""
    safe = dict(kwargs)
    if "secret_value" in safe:
        safe["secret_value"] = "***"
    return {"tool": "keyvault_secret", "status": "ok", "kwargs": safe}

def tool_observability_log(**kwargs):
    """Observability/logging placeholder."""
    return {"tool": "observability_log", "status": "ok", "kwargs": kwargs}

TOOLS = {
    "tool_storage_blob": tool_storage_blob,
    "tool_search_query": tool_search_query,
    "tool_search_admin": tool_search_admin,
    "tool_keyvault_secret": tool_keyvault_secret,
    "tool_observability_log": tool_observability_log,
}
print("Tools registered:", list(TOOLS.keys()))

In [ ]:
# %% [AGENTS]
import json, re
from typing import List

class SKAgent:
    def __init__(self, kernel, name: str, skills: List[str] | None = None, system: str | None = None):
        self.kernel = kernel
        self.name = name
        self.system_message = system or f"You are the '{name}' capability agent."
        self.skills = skills or []

    async def run(self, user_text: str) -> str:
        try:
            result = await self.kernel.invoke_prompt(self.system_message + "\n\nUser: " + user_text)
            return str(result)
        except Exception as e:
            return f"[{self.name} stub] SK call not available yet. Error: {e}"

    def available_tools(self):
        return [t for t in self.skills if t in TOOLS]

    def call(self, tool_name: str, **kwargs):
        fn = TOOLS.get(tool_name)
        if not fn:
            raise ValueError(f"Tool not found: {tool_name}")
        return fn(**kwargs)

def _slugify(label: str) -> str:
    return re.sub(r'[^a-z0-9]+', '_', label.lower()).strip('_')

AGENTS = {}
WIRES = {}

_AGENT_NAMES = ["App Service (Web/API)", "Container Apps Env", "Azure Functions", "Storage Account: stproddata", "Blob (Data Lake)", "Queues", "Tables", "File Shares", "Search Service", "Index: docs", "Vector Index: embeddings", "Skillset / ETL", "Key Vault", "Managed Identity", "Private Endpoint: Search", "Private Endpoint: Storage", "Azure Monitor / Insights"]
_DEFAULT_WIRES = {"App Service (Web/API)": ["tool_storage_blob", "tool_search_query"], "Container Apps Env": ["tool_search_query", "tool_storage_blob"], "Azure Functions": ["tool_search_admin", "tool_storage_blob"], "Storage Account: stproddata": [], "Blob (Data Lake)": [], "Queues": [], "Tables": [], "File Shares": [], "Search Service": ["tool_search_query"], "Index: docs": [], "Vector Index: embeddings": [], "Skillset / ETL": ["tool_search_admin"], "Key Vault": ["tool_keyvault_secret"], "Managed Identity": [], "Private Endpoint: Search": [], "Private Endpoint: Storage": [], "Azure Monitor / Insights": ["tool_observability_log"]}

for _name in __AGENT_NAMES:
    skills = _DEFAULT_WIRES.get(_name, [])
    agent = SKAgent(kernel, _name, skills=skills)
    AGENTS[_slugify(_name)] = agent
    WIRES[_name] = {"tools": skills}

print("Agents created:", list(AGENTS.keys()))
print("Wiring entries:", len(WIRES))


In [ ]:
# %% [DEMO]
import asyncio

first_key = next(iter(AGENTS))
agent = AGENTS[first_key]

print("Agent:", agent.name)
print("Tools:", agent.available_tools())

# Try a tool if present
if agent.available_tools():
    tname = agent.available_tools()[0]
    print("Tool demo:", tname, "->", agent.call(tname, example="value"))

async def _demo():
    try:
        return await agent.run("Say hello and confirm you are wired via SK.")
    except Exception as e:
        return f"[demo] invoke failed: {e}"

try:
    loop = asyncio.get_running_loop()
    try:
        result = await _demo()
    except SyntaxError:
        import nest_asyncio
        nest_asyncio.apply()
        result = loop.run_until_complete(_demo())
except RuntimeError:
    result = asyncio.run(_demo())

print("\nLLM demo:\n", result)